In [17]:
pip install tensorflow matplotlib seaborn numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tensorflow in c:\users\owner\appdata\local\programs\python\python311\lib\site-packages (2.15.0)




[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import tensorflow as tf
import numpy as np
import datetime
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
# ✅ List available devices
print("Available devices:", tf.config.list_physical_devices())

# ✅ Set training parameters
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32  
EPOCHS = 50  # Increased for better learning
DIRECTORY = "datasets"

# ✅ Data Augmentation - More transformations for underrepresented classes
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize images
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [20]:
# ✅ Load datasets with augmentation
train_dataset = train_datagen.flow_from_directory(
    DIRECTORY,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    subset="training",
    seed=123
)

validation_dataset = train_datagen.flow_from_directory(
    DIRECTORY,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    subset="validation",
    seed=123
)

Found 13143 images belonging to 6 classes.
Found 3283 images belonging to 6 classes.


In [21]:
# ✅ Get class names & count
class_names = list(train_dataset.class_indices.keys())
num_classes = len(class_names)
print("Class Names:", class_names)

Class Names: ['Alternaria_Leaf_Spot', 'Bacterial_Soft_Rot', 'Black_Rot', 'Downy_Mildew', 'Healthy_Pechay', 'Undefined']


In [22]:
#Data Checks
class_names

['Alternaria_Leaf_Spot',
 'Bacterial_Soft_Rot',
 'Black_Rot',
 'Downy_Mildew',
 'Healthy_Pechay',
 'Undefined']

In [23]:
# ✅ Load Pretrained Model (MobileNetV3)
base_model = MobileNetV3Large(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze for initial training

In [24]:
# Custom Classification Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [25]:
# ✅ Learning Rate Decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9
)

In [26]:
# ✅ Compile Model
model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [27]:
# ✅ TensorBoard for Monitoring
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [28]:
# ✅ Model Checkpoint - Save Best Model
checkpoint_callback = ModelCheckpoint(
    filepath="best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)


# ✅ Early Stopping - Prevent Overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [ ]:
# ✅ Train Model
history = model.fit(
    train_dataset,
    epochs=15,  # Initial Training
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback, checkpoint_callback, early_stopping]
)

Epoch 1/15
411/411 [==============================] - ETA: 0s - loss: 1.0660 - accuracy: 0.6222
Epoch 1: val_accuracy improved from -inf to 0.47304, saving model to best_model.h5


c:\Users\Owner\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


411/411 [==============================] - 1653s 4s/step - loss: 1.0660 - accuracy: 0.6222 - val_loss: 1.4917 - val_accuracy: 0.4730
Epoch 2/15
411/411 [==============================] - ETA: 0s - loss: 0.9044 - accuracy: 0.6903 
Epoch 2: val_accuracy improved from 0.47304 to 0.58544, saving model to best_model.h5
411/411 [==============================] - 7736s 19s/step - loss: 0.9044 - accuracy: 0.6903 - val_loss: 1.4635 - val_accuracy: 0.5854
Epoch 3/15
278/411 [===================>..........] - ETA: 1:54:51 - loss: 0.8618 - accuracy: 0.6993

In [ ]:
# ✅ Unfreeze More Layers for Fine-Tuning
base_model.trainable = True
for layer in base_model.layers[:50]:  # Keep first 50 layers frozen
    layer.trainable = False

# ✅ Recompile Model for Fine-Tuning
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])


In [ ]:
# ✅ Continue Training with Fine-Tuning
history_fine = model.fit(
    train_dataset,
    epochs=30,  # Fine-tuning for better accuracy
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback, checkpoint_callback, early_stopping]
)


Epoch 1/5
411/411 [==============================] - 2972s 7s/step - loss: 0.0343 - accuracy: 0.9886 - val_loss: 0.0686 - val_accuracy: 0.9787
Epoch 2/5


KeyboardInterrupt: 

In [ ]:
# ✅ Save Final Model
model.save("final_model.h5")

# ✅ Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("bokchoymodel.tflite", "wb") as f:
    f.write(tflite_model)

# ✅ Save Class Labels
with open("petchay_labels.txt", "w") as f:
    for label in class_names:
        f.write(label + "\n")

In [ ]:
# ✅ Load Test Image & Predict
img_path = "C:/Users/Owner/Desktop/aimodeltrain/datasets/Undefined/undefined (2).jpg"

img = tf.keras.utils.load_img(img_path, target_size=IMAGE_SIZE)
img_array = tf.keras.utils.img_to_array(img) / 255.0
img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions)
print("This image most likely belongs to:", class_names[np.argmax(score)])


In [ ]:
# ✅ Evaluate Model & Show F1-Score
y_true = []
y_pred = []

for i in range(len(validation_dataset)):
    images, labels = validation_dataset[i]
    preds = model.predict(images)
    y_true.extend(labels)
    y_pred.extend(np.argmax(preds, axis=1))

report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
print(report)
